### Initialize notebook variables

Configures everything that's needed for deployment. 

**Modify entries under _1) User-defined parameters_ and _2) Define the APIs and their operations and policies_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.SIMPLE_APIM
tags        = ['general']

# 2) Define the APIs and their operations and policies

# API 1
api1_get = GET_APIOperation('This is a GET for API 1')
api1_post = POST_APIOperation('This is a POST for API 1')
api1 = API('API1', 'API 1', '/api1', 'This is API 1', operations = [api1_get, api1_post], tags = tags)

# API 2
api2_post = POST_APIOperation('This is a POST for API 2')
api2 = API('API2', 'API 2', '/api2', 'This is API 2', operations = [api2_post], tags = tags)

# Request Headers
pol_request_headers_get = utils.read_policy_xml(REQUEST_HEADERS_XML_POLICY_PATH)
request_headers_get     = GET_APIOperation('Gets the request headers for the current request and returns them. Great for troubleshooting.', pol_request_headers_get)
request_headers         = API('requestheaders', 'Request Headers', '/request-headers', 'API for request headers', operations = [request_headers_get], tags = tags)

# APIs Array
apis: List[API] = [api1, api2, request_headers]

utils.print_ok('Notebook initialized')

### Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
# Build the bicep parameters
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]}
}

# Create the deployment helper and deploy the sample
nb_helper = utils.NotebookHelper('general', utils.get_infra_rg_name(deployment, index), rg_location, deployment, [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM], index = index)
output = nb_helper.deploy_sample(bicep_parameters)

if output.success:
    # Extract deployment outputs for testing
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM Gateway URL')
    apim_apis = output.getJson('apiOutputs', 'APIs')

    print(f"✅ Sample deployment completed successfully!")
else:
    print("❌ Sample deployment failed!")
    raise SystemExit(1)

### Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
from apimrequests import ApimRequests
from apimtesting import ApimTesting

# Initialize testing framework
tests = ApimTesting("General Sample Tests", 'general', nb_helper.deployment)
api_subscription_key = apim_apis[2]['subscriptionPrimaryKey']

# Check if the infrastructure uses Azure Front Door
utils.print_message('Checking infrastructure endpoint...', blank_above = True)
afd_endpoint_url = utils.get_frontdoor_url(nb_helper.deployment, nb_helper.rg_name)

if afd_endpoint_url:
    # Test via Azure Front Door
    reqsAfd = ApimRequests(afd_endpoint_url, api_subscription_key)
    output = reqsAfd.singleGet('/request-headers', msg = 'Calling Request Headers API via Azure Front Door. Expect 200.')
    tests.verify('Host:' in output, True)
else:
    # Test via API Management directly
    reqsApim = ApimRequests(apim_gateway_url, api_subscription_key)
    output = reqsApim.singleGet('/request-headers', msg = 'Calling Request Headers API via API Management Gateway. Response codes 200 and 403 are both valid.')
    tests.verify('Host:' in output, True)

tests.print_summary()
utils.print_ok('All done!')